<a href="https://colab.research.google.com/github/ghostfm3/ksks/blob/master/covariance_matrix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get -q -y install sudo file mecab libmecab-dev mecab-ipadic-utf8 git curl python-mecab > /dev/null
!git clone --depth 1 https://github.com/neologd/mecab-ipadic-neologd.git > /dev/null 
!echo yes | mecab-ipadic-neologd/bin/install-mecab-ipadic-neologd -n > /dev/null 2>&1
!pip install mecab-python3 > /dev/null
!echo mecab-config --dicdir"/mecab-ipadic-neologd"

Cloning into 'mecab-ipadic-neologd'...
remote: Enumerating objects: 75, done.
remote: Counting objects: 100% (75/75), done.
remote: Compressing objects: 100% (74/74), done.
remote: Total 75 (delta 5), reused 54 (delta 0), pack-reused 0
Unpacking objects: 100% (75/75), done.
mecab-config --dicdir/mecab-ipadic-neologd


In [2]:
pip install unidic-lite

     |████████████████████████████████| 47.3MB 67kB/s 
  Created wheel for unidic-lite: filename=unidic_lite-1.0.7-cp36-none-any.whl size=47556594 sha256=cef39ff9e76e90003db808143a75432bb334d598043c34a4f54d6018d354f804
  Stored in directory: /root/.cache/pip/wheels/a8/82/7d/086724645e33a575aafd0b1dae2835c37d2c00c6a0a96ee3a0
Successfully built unidic-lite


In [3]:
from google.colab import files
uploaded = files.upload()

Saving 吾輩は猫である_ルビなし.txt to 吾輩は猫である_ルビなし.txt


In [4]:
import numpy as np
import sys
import MeCab
import codecs
import logging
sys.path.append('..')
def preprocess(text):
  text = text.lower()
  text = text.replace('.', ' .')
  words = text.split(' ')

  word_to_id = {}
  id_to_word = {}
  for word in words:
    if word not in word_to_id:
      new_id = len(word_to_id)
      word_to_id[word] = new_id
      id_to_word[new_id] = word 
  
  corpus = np.array([word_to_id[w] for w in words])
  return corpus, word_to_id, id_to_word

tagger = MeCab.Tagger('-Owakati')
text1 = codecs.open('吾輩は猫である_ルビなし.txt', 'r', 'cp932', "ignore").read().splitlines()
#print(' '.join(text1))

result = tagger.parse(''.join(text1))
text = result

corpus, word_to_id, id_to_word = preprocess(text)
print(corpus)

print(id_to_word)

[    0     1     2 ...  1601     6 13769]
{0: '\u3000', 1: '吾輩', 2: 'は', 3: '猫', 4: 'で', 5: 'ある', 6: '。', 7: '名前', 8: 'まだ', 9: '無い', 10: 'どこ', 11: '生れ', 12: 'た', 13: 'か', 14: 'と', 15: 'んと', 16: '見当', 17: 'が', 18: 'つか', 19: 'ぬ', 20: '何', 21: 'も', 22: '薄暗い', 23: 'じめじめ', 24: 'し', 25: '所', 26: 'ニャーニャー', 27: '泣い', 28: 'て', 29: 'い', 30: '事', 31: 'だけ', 32: '記憶', 33: 'いる', 34: 'ここ', 35: '始め', 36: '人間', 37: 'いう', 38: 'もの', 39: 'を', 40: '見', 41: 'しかも', 42: 'あと', 43: '聞く', 44: 'それ', 45: '書生', 46: '中', 47: '一番', 48: '獰悪', 49: 'な', 50: '種族', 51: 'あっ', 52: 'そう', 53: 'だ', 54: 'この', 55: 'の', 56: '時々', 57: '我々', 58: '捕え', 59: '煮', 60: '食う', 61: '話', 62: 'しかし', 63: 'その', 64: '当時', 65: '考', 66: 'なかっ', 67: 'から', 68: '別段', 69: '恐しい', 70: '思わ', 71: 'ただ', 72: '彼', 73: '掌', 74: 'に', 75: '載せ', 76: 'られ', 77: 'スー', 78: '持ち上げ', 79: '時', 80: 'フワフワ', 81: '感じ', 82: 'ばかり', 83: '上', 84: '少し', 85: '落ちつい', 86: '顔', 87: 'いわゆる', 88: '始', 89: 'あろう', 90: '妙', 91: '思っ', 92: '今', 93: '残っ', 94: '第', 95: '一', 96: '毛', 97: 'もっ',

In [5]:
import sys 
sys.path.append('..')
import numpy as np
from sklearn.decomposition import PCA
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import MeCab
import codecs

def preprocess(text):
    text = text.lower()
    text = text.replace('.', ' .')
    words = text.split(' ')

    word_to_id = {}
    id_to_word = {}
    for word in words:
        if word not in word_to_id:
            new_id = len(word_to_id)
            word_to_id[word] = new_id
            id_to_word[new_id] = word

    corpus = np.array([word_to_id[w] for w in words])

    return corpus, word_to_id, id_to_word

def create_co_matrix(corpus, vocab_size, window_size=1):
    '''共起行列の作成
    :param corpus: コーパス（単語IDのリスト）
    :param vocab_size:語彙数
    :param window_size:ウィンドウサイズ（ウィンドウサイズが1のときは、単語の左右1単語がコンテキスト）
    :return: 共起行列
    '''
    corpus_size = len(corpus)
    co_matrix = np.zeros((vocab_size, vocab_size), dtype=np.int32)

    for idx, word_id in enumerate(corpus):
        for i in range(1, window_size + 1):
            left_idx = idx - i
            right_idx = idx + i

            if left_idx >= 0:
                left_word_id = corpus[left_idx]
                co_matrix[word_id, left_word_id] += 1

            if right_idx < corpus_size:
                right_word_id = corpus[right_idx]
                co_matrix[word_id, right_word_id] += 1

    return co_matrix

def ppmi(C, verbose=False, eps = 1e-8):
    '''PPMI（正の相互情報量）の作成
    :param C: 共起行列
    :param verbose: 進行状況を出力するかどうか
    :return:
    '''
    M = np.zeros_like(C, dtype=np.float32)
    N = np.sum(C)
    S = np.sum(C, axis=0)
    total = C.shape[0] * C.shape[1]
    cnt = 0

    for i in range(C.shape[0]):
        for j in range(C.shape[1]):
            pmi = np.log2(C[i, j] * N / (S[j]*S[i]) + eps)
            M[i, j] = max(0, pmi)

            if verbose:
                cnt += 1
                if cnt % (total//100 + 1) == 0:
                    print('%.1f%% done' % (100*cnt/total))
    return M

tagger = MeCab.Tagger('-Owakati')
text1 = codecs.open('吾輩は猫である_ルビなし.txt', 'r', 'cp932', "ignore").read().splitlines()
#print(' '.join(text1))

result = tagger.parse(''.join(text1))
text = result


corpus, word_to_id, id_to_word = preprocess(text)
vocab_size = len(word_to_id)
C = create_co_matrix(corpus, vocab_size)
W = ppmi(C)

np.set_printoptions(precision=3)
print('covariance matrix')
print(C)
print('PPMI')
print(W)

covariance matrix
[[  8  32   0 ...   0   0   0]
 [ 32   0 214 ...   0   0   0]
 [  0 214   0 ...   0   0   0]
 ...
 [  0   0   0 ...   0   0   0]
 [  0   0   0 ...   0   0   0]
 [  0   0   0 ...   0   0   0]]
PPMI
[[2.327 4.091 0.    ... 0.    0.    0.   ]
 [4.091 0.    2.84  ... 0.    0.    0.   ]
 [0.    2.84  0.    ... 0.    0.    0.   ]
 ...
 [0.    0.    0.    ... 0.    0.    0.   ]
 [0.    0.    0.    ... 0.    0.    0.   ]
 [0.    0.    0.    ... 0.    0.    0.   ]]
